# 实验参数设置

In [1]:
from FederatedLearning.learner import CNNModel

clients_num = 10 # 客户端总数
rotation_cycle = 5 # 角色轮换间隔轮数
max_round = 10 # 联邦学习轮数
model_class = CNNModel # 选用CNN模型

## 数据切分

In [2]:
import data_split
# 使用data_split模块中的函数获取PyTorch数据集
train_dataset = data_split.get_mnist_pytorch_dataset(train=True)
test_dataset = data_split.get_mnist_pytorch_dataset(train=False)
# test
# 使用正确的数据集对象创建数据加载器
client_dataloaders = data_split.create_client_dataloaders(train_dataset, clients_num // 2, 64, num_workers=clients_num) * 2
client_test_loader = data_split.create_client_dataloaders(test_dataset, clients_num // 2, 64) * 2

2025-10-19 17:20:04,936 - DGS_BCFL - INFO - 
切分5份数据集...
2025-10-19 17:20:04,938 - DGS_BCFL - INFO -  1 数据准备完成，样本数: 12000
2025-10-19 17:20:04,939 - DGS_BCFL - INFO -  2 数据准备完成，样本数: 12000
2025-10-19 17:20:04,940 - DGS_BCFL - INFO -  3 数据准备完成，样本数: 12000
2025-10-19 17:20:04,941 - DGS_BCFL - INFO -  4 数据准备完成，样本数: 12000
2025-10-19 17:20:04,942 - DGS_BCFL - INFO -  5 数据准备完成，样本数: 12000
2025-10-19 17:20:04,943 - DGS_BCFL - INFO - 
切分5份数据集...
2025-10-19 17:20:04,943 - DGS_BCFL - INFO -  1 数据准备完成，样本数: 2000
2025-10-19 17:20:04,944 - DGS_BCFL - INFO -  2 数据准备完成，样本数: 2000
2025-10-19 17:20:04,945 - DGS_BCFL - INFO -  3 数据准备完成，样本数: 2000
2025-10-19 17:20:04,946 - DGS_BCFL - INFO -  4 数据准备完成，样本数: 2000
2025-10-19 17:20:04,947 - DGS_BCFL - INFO -  5 数据准备完成，样本数: 2000


## 开始训练

In [3]:
from owner import Owner
from client import Client

# 初始化管理者，并获得初始化字典
owner = Owner(rotation_cycle=1, model_class=model_class)
main_dict = owner.get_main_dict()

# 初始化客户端以及身份
clients = [owner]
for i in range(clients_num):
    client_name = f"client_{i + 1}"
    client = Client(epochs=2, client_name=client_name, data_loader=client_dataloaders[i],
                    test_loader=client_test_loader[i], ModelClass=model_class, main_dict=main_dict)
    owner.join(client_name)
    clients.append(client)

import threading

for _ in range(max_round):
    t = [threading.Thread(target=client.run) for client in clients]
    _ = [i.start() for i in t]
    _ = [i.join() for i in t]


# 保存实验过程和结果
import pickle
with open("main_dict.pkl", "wb") as f:
    pickle.dump(main_dict, f)

2025-10-19 17:20:07,719 - DGS_BCFL - INFO - Owner开始初始化 角色
2025-10-19 17:20:07,720 - DGS_BCFL - INFO - [client_1]  round 1 等待角色分配...
2025-10-19 17:20:07,720 - DGS_BCFL - INFO - 初始化完成 {'role': [{'client_1': 'aggregator', 'client_2': 'validator', 'client_3': 'validator', 'client_4': 'validator', 'client_5': 'learner', 'client_6': 'learner', 'client_7': 'learner', 'client_8': 'learner', 'client_9': 'learner', 'client_10': 'learner'}], 'global_model': ['./global_model/init_global_model'], 'client_gradients': [], 'votes': [], 'contribution': {'client_1': 0, 'client_2': 0, 'client_3': 0, 'client_4': 0, 'client_5': 0, 'client_6': 0, 'client_7': 0, 'client_8': 0, 'client_9': 0, 'client_10': 0}, 'global_accuracy_history': [], 'contribution_history': []}
2025-10-19 17:20:07,721 - DGS_BCFL - INFO - [client_2] 当前轮次角色: validator
2025-10-19 17:20:07,721 - DGS_BCFL - INFO - [client_3] 当前轮次角色: validator
2025-10-19 17:20:07,722 - DGS_BCFL - INFO - [client_4] 当前轮次角色: validator
2025-10-19 17:20:07,722 - D

## 计算结果可视化

In [ ]:
# 读取实验结果
main_dict = pickle.load(open("main_dict.pkl", "rb"))
# 进行可视化